# Van311 Service Request 
## Data Cleaning for Mapping Visualization

### 1. Import Libraries and Datasets

In [ ]:
# Import libraries needed for this lab
import pandas as pd
import altair as alt

In [14]:
#Import service request datasets and merge them together
#Link (2022-2024): https://opendata.vancouver.ca/explore/dataset/3-1-1-service-requests/.
#Link (2009-2021) -I only extracted 2017-2021: https://opendata.vancouver.ca/explore/dataset/3-1-1-service-requests-2009-2021/
service_requests_2224 = pd.read_csv("3-1-1-service-requests.csv",sep = ";")
service_requests_1721 = pd.read_csv("3-1-1-service-requests-2017-2021.csv" ,sep = ";")
service_requests = pd.concat([service_requests_1721, service_requests_2224], ignore_index=True)
service_requests.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896009 entries, 0 to 1896008
Data columns (total 13 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   Department                      object 
 1   Service request type            object 
 2   Status                          object 
 3   Closure reason                  object 
 4   Service request open timestamp  object 
 5   Service request close date      object 
 6   Last modified timestamp         object 
 7   Address                         object 
 8   Local area                      object 
 9   Channel                         object 
 10  Latitude                        float64
 11  Longitude                       float64
 12  geom                            object 
dtypes: float64(2), object(11)
memory usage: 188.1+ MB


### 2. Cleaning Data

* changed name of Arbutus Ridge to "Arbutus-Ridge" to match mapping shapefile
* convert time data to correct format
* calculate service processing time (difference between request open and close date)

In [15]:
service_requests.loc[service_requests["Local area"] == "Arbutus Ridge", "Local area"] = "Arbutus-Ridge"
service_requests = service_requests.assign(service_date = pd.to_datetime(service_requests['Service request open timestamp'], utc =True))
service_requests.describe(include='all')

,Department,Service request type,Status,Closure reason,Service request open timestamp,Service request close date,Last modified timestamp,Address,Local area,Channel,Latitude,Longitude,geom,service_date
count,1896009,1896009,1896009,1878900,1896009,1878736,1896009,778551,1559629,1896009,774281.000000,774281.000000,774281,1896009
unique,75,247,2,16,1818811,2855,1427976,115737,22,10,NaN,NaN,302958,NaN
top,ENG - Sanitation Services,Abandoned Non-Recyclables-Small Case,Close,Service provided,2021-10-31T20:13:04-07:00,2022-08-09,2021-11-15T17:32:11-08:00,2000 W GEORGIA ST,Downtown,Phone,NaN,NaN,"49.27191, -123.11596",NaN
freq,495392,98236,1878736,1023722,11,6050,58,3617,201792,882867,NaN,NaN,5289,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.258431,-123.104874,NaN,2021-03-01 18:24:50.347681024+00:00
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.098370,-123.385073,NaN,2017-01-01 11:03:39+00:00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.241700,-123.133360,NaN,2019-05-04 21:37:42+00:00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.263120,-123.105280,NaN,2021-05-04 20:47:17+00:00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.277821,-123.071230,NaN,2023-01-23 22:27:41+00:00
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.314098,-123.013490,NaN,2024-10-26 04:57:36+00:00


In [16]:
service_requests['Service request open timestamp'] = pd.to_datetime(service_requests['Service request open timestamp'], utc =True)
service_requests['Service request close date'] = pd.to_datetime(service_requests['Service request close date'], utc =True)
service_requests['Year'] = service_requests['Service request open timestamp'].dt.year
#find the difference in time between open and close date of request and round to the nearest number of days
service_requests['request_time'] = (service_requests['Service request close date'] - service_requests['Service request open timestamp']).dt.round('1d').dt.days
service_requests.head()

,Department,Service request type,Status,Closure reason,Service request open timestamp,Service request close date,Last modified timestamp,Address,Local area,Channel,Latitude,Longitude,geom,service_date,Year,request_time
0,ENG - Sanitation Services,Abandoned Non-Recyclables-Small Case,Close,Service provided,2017-08-22 16:43:37+00:00,2017-09-01 00:00:00+00:00,2017-09-01T15:19:43-07:00,387 W 26TH AV,Riley Park,Phone,49.24817,-123.11199,"49.24817, -123.11199",2017-08-22 16:43:37+00:00,2017,9.0
1,DBL - Services Centre,Building and Development Inquiry Case,Close,Service provided,2017-08-22 17:01:38+00:00,2017-08-29 00:00:00+00:00,2017-08-29T10:19:55-07:00,NaN,Riley Park,Phone,NaN,NaN,NaN,2017-08-22 17:01:38+00:00,2017,6.0
2,ENG - Streets Operations,Street Repair Case,Close,Service provided,2017-08-22 17:05:11+00:00,2017-08-24 00:00:00+00:00,2017-08-24T09:06:27-07:00,2596 W 7TH AV,Kitsilano,Phone,49.26549,-123.16441,"49.26549, -123.16441",2017-08-22 17:05:11+00:00,2017,1.0
3,ENG - Sewer Operations,Sewer Construction Concern Case,Close,Service provided,2017-08-22 17:07:55+00:00,2017-08-23 00:00:00+00:00,2017-08-23T15:27:28-07:00,777 E 32ND AV,Kensington-Cedar Cottage,Phone,49.24194,-123.08858,"49.24194, -123.08858",2017-08-22 17:07:55+00:00,2017,0.0
4,ENG - Sanitation Services,Missed Garbage Bin Pickup Case,Close,Service provided,2017-08-22 17:08:06+00:00,2017-08-23 00:00:00+00:00,2017-08-23T14:57:28-07:00,NaN,Arbutus-Ridge,Phone,NaN,NaN,NaN,2017-08-22 17:08:06+00:00,2017,0.0


### 3. Aggregation only by Local Area (combines all years 2017-2024)

In [17]:
#aggregate counts by Local area
service_request_count = service_requests.loc[:, ['Local area']].groupby(by ='Local area').size().reset_index(name='request_counts')
servive_request_count = service_request_count.loc[service_requests["Local area"] == "Arbutus Ridge", "Local area"] = "Arbutus-Ridge"
#add a 'name' column to allow linking to geographic file
service_request_count = service_request_count.assign(name = service_request_count['Local area'])
service_request_count

,Local area,request_counts,name
0,Arbutus-Ridge,31687,Arbutus-Ridge
1,Downtown,201792,Downtown
2,Dunbar-Southlands,56513,Dunbar-Southlands
3,Fairview,68585,Fairview
4,Grandview-Woodland,92072,Grandview-Woodland
5,Hastings-Sunrise,86220,Hastings-Sunrise
6,Kensington-Cedar Cottage,121122,Kensington-Cedar Cottage
7,Kerrisdale,35682,Kerrisdale
8,Killarney,44683,Killarney
9,Kitsilano,104507,Kitsilano


### 4. Import Vancouver Map Shapefile

In [18]:
#Creating a chloropleth map for tree density
#geojson url for map
url_geojson = 'https://raw.githubusercontent.com/UBC-MDS/exploratory-data-viz/main/data/local-area-boundary.geojson'
#convert to topo json format
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))
data_geojson_remote
#base vancouver map
vancouver_map = alt.Chart(data_geojson_remote).mark_geoshape(
    color = 'gray', opacity= 0.5, stroke='white'
).encode().project(type='identity', reflectY=True)

### 5. Map of All Requests (2017-2024 combined)

In [70]:
map_title = alt.TitleParams("Number of Requests Per Neighbourhood (2017-2024)")
simple_request_map  = alt.Chart(data_geojson_remote, title = map_title).mark_geoshape().transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(service_request_count, 'name', ['request_counts' ,'name', 'neighbourhood_name'])).encode(
    color= alt.Color('request_counts:Q', title = "Request Counts",
                    scale=alt.Scale(scheme = 'reds', zero = False)),
    tooltip= [alt.Tooltip('name:N', title = 'Neighbourhood'),
             alt.Tooltip('request_counts:Q', title = 'Request_Counts')]
   ).project(type='identity', reflectY=True)

simple_request_map

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

alt.Chart(...)

### 6. Aggregate by Local Area and Year and Extract CSVs for use in Tableau

In [72]:
#aggregate counts by Local area
service_request_count2 = service_requests.loc[:, ['Local area', 'Year']].groupby(by =['Local area', 'Year']).size().reset_index(name='request_counts')
servive_request_count2 = service_request_count2.loc[service_requests["Local area"] == "Arbutus Ridge", "Local area"] = "Arbutus-Ridge"
#add a 'name' column to allow linking to geographic file
service_request_count2 = service_request_count2.assign(name = service_request_count2['Local area'])

In [ ]:
#Export to csv 
service_request_count2.to_csv('service_request_volume_aggregation.csv')

**Find total requests in 2023**

In [73]:
service_request_2023 = service_request_count2[service_request_count2['Year']== 2023]
service_request_2023[['request_counts']].sum()

request_counts    208934
dtype: int64

In [74]:
top_2023 = service_request_2023.groupby('Local area')['request_counts'].sum().idxmax()
top_2023

'Downtown'

**Find total requests in 2017**

In [75]:
service_request_2017 = service_request_count2[service_request_count2['Year']== 2017]
service_request_2017[['request_counts']].sum()

request_counts    183644
dtype: int64

In [57]:
#aggregate counts by Local area
service_request_time = service_requests.loc[:, ['Local area', 'Year', 'request_time']].groupby(by =['Local area', 'Year']).mean().reset_index()
servive_request_time = service_request_time.loc[service_requests["Local area"] == "Arbutus Ridge", "Local area"] = "Arbutus-Ridge"
#add a 'name' column to allow linking to geographic file
service_request_time = service_request_time.assign(name = service_request_time['Local area'])
service_request_time.to_csv('service_request_processing_time_aggregation.csv')

In [ ]:
#extract aggregated (by year and Local area) processing time data to csv
service_request_time.to_csv('service_request_processing_time_aggregation.csv')

In [28]:
time_means = service_requests.loc[:, ['Year', 'request_time']].groupby(by =['Year']).mean().reset_index()
time_means

,Year,request_time
0,2017,22.857895
1,2018,37.663364
2,2019,22.241714
3,2020,13.500152
4,2021,10.310364
5,2022,8.188322
6,2023,8.046006
7,2024,5.284504


In [62]:
service_requests.describe(include = 'all')

,Department,Service request type,Status,Closure reason,Service request open timestamp,Service request close date,Last modified timestamp,Address,Local area,Channel,Latitude,Longitude,geom,service_date,Year,request_time
count,1896009,1896009,1896009,1878900,1896009,1878736,1896009,778551,1559629,1896009,774281.000000,774281.000000,774281,1896009,1.896009e+06,1.878736e+06
unique,75,247,2,16,NaN,NaN,1427976,115737,22,10,NaN,NaN,302958,NaN,NaN,NaN
top,ENG - Sanitation Services,Abandoned Non-Recyclables-Small Case,Close,Service provided,NaN,NaN,2021-11-15T17:32:11-08:00,2000 W GEORGIA ST,Downtown,Phone,NaN,NaN,"49.27191, -123.11596",NaN,NaN,NaN
freq,495392,98236,1878736,1023722,NaN,NaN,58,3617,201792,882867,NaN,NaN,5289,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,2021-03-01 18:24:50.347681024+00:00,2021-03-08 02:37:55.810970624+00:00,NaN,NaN,NaN,NaN,49.258431,-123.104874,NaN,2021-03-01 18:24:50.347681024+00:00,2.020675e+03,1.521850e+01
min,NaN,NaN,NaN,NaN,2017-01-01 11:03:39+00:00,2017-01-01 00:00:00+00:00,NaN,NaN,NaN,NaN,49.098370,-123.385073,NaN,2017-01-01 11:03:39+00:00,2.017000e+03,-2.000000e+00
25%,NaN,NaN,NaN,NaN,2019-05-04 21:37:42+00:00,2019-05-20 00:00:00+00:00,NaN,NaN,NaN,NaN,49.241700,-123.133360,NaN,2019-05-04 21:37:42+00:00,2.019000e+03,-1.000000e+00
50%,NaN,NaN,NaN,NaN,2021-05-04 20:47:17+00:00,2021-05-14 00:00:00+00:00,NaN,NaN,NaN,NaN,49.263120,-123.105280,NaN,2021-05-04 20:47:17+00:00,2.021000e+03,2.000000e+00
75%,NaN,NaN,NaN,NaN,2023-01-23 22:27:41+00:00,2023-01-20 00:00:00+00:00,NaN,NaN,NaN,NaN,49.277821,-123.071230,NaN,2023-01-23 22:27:41+00:00,2.023000e+03,6.000000e+00
max,NaN,NaN,NaN,NaN,2024-10-26 04:57:36+00:00,2024-10-25 00:00:00+00:00,NaN,NaN,NaN,NaN,49.314098,-123.013490,NaN,2024-10-26 04:57:36+00:00,2.024000e+03,1.791000e+03


In [60]:
#extract unaggregated service request processing time to csv
time_export = service_requests.loc[:, ['Local area', 'Year', 'request_time']]
time_export.to_csv('service_request_processing_time.csv')

### 7. Mapping of Number of Service Requests

In [85]:
#MAP 2023
service_request_2023 = service_request_count2[service_request_count2['Year']== 2023]
service_request_2023

map_title = alt.TitleParams("Requests in 2023 (Total N=208,934)")
request_map_2023  = alt.Chart(data_geojson_remote, title = map_title).mark_geoshape().transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(service_request_2023, 'name', ['request_counts' ,'name', 'neighbourhood_name'])).encode(
    color= alt.Color('request_counts:Q', title = "Request Counts",
                    scale=alt.Scale(scheme = 'reds', domain=[2000,25000])),
    tooltip= [alt.Tooltip('name:N', title = 'Neighbourhood'),
             alt.Tooltip('request_counts:Q', title = 'Request_Counts')]
   ).project(type='identity', reflectY=True).properties(
    height = 400, width =400)


request_map_2023 =request_map_2023 + request_map_2023.mark_text(align = 'left', dx=2, dy=-100).encode(text = alt.condition(alt.datum.name == top_2023,
                                                                          alt.value('Downtown (N = 25,242)'),
                                                                         alt.value('')))

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

In [86]:
#MAP 2017
service_request_2017 = service_request_count2[service_request_count2['Year']== 2017]
service_request_2017

map_title = alt.TitleParams("Requests in 2017 (Total N=183,644)")
request_map_2017  = alt.Chart(data_geojson_remote, title = map_title).mark_geoshape().transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(service_request_2017, 'name', ['request_counts' ,'name', 'neighbourhood_name'])).encode(
    color= alt.Color('request_counts:Q', title = "Request Counts",
                    scale=alt.Scale(scheme = 'reds', domain=[2000,25000])),
    tooltip= [alt.Tooltip('name:N', title = 'Neighbourhood'),
             alt.Tooltip('request_counts:Q', title = 'Request_Counts')]
   ).project(type='identity', reflectY=True).properties(
    height = 400, width =400)

request_map_2017 =request_map_2017 + request_map_2017.mark_text(align = 'left', dx=2, dy=-100).encode(text = alt.condition(alt.datum.name == top_2023,
                                                                          alt.value('Downtown (N = 19,048)'),
                                                                         alt.value('')))


request_map_2017

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

alt.LayerChart(...)

In [87]:
request_map_2017 | request_map_2023

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

alt.HConcatChart(...)

In [88]:
#MAP 2023
service_time_2023 = service_request_time[service_request_time['Year']== 2023]
service_time_2023

map_title = alt.TitleParams("2023")
request_time_2023  = alt.Chart(data_geojson_remote, title = map_title).mark_geoshape().transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(service_time_2023, 'name', ['request_time' ,'name', 'neighbourhood_name'])).encode(
    color= alt.Color('request_time:Q', title = "Average days",
                    scale=alt.Scale(scheme = 'blues', zero=False)),
    tooltip= [alt.Tooltip('name:N', title = 'Neighbourhood'),
             alt.Tooltip('request_time:Q', title = 'Request_Time (days)')]
   ).project(type='identity', reflectY=True).properties(
    height = 00, width =500)

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

In [89]:
#MAP 2017
service_time_2017 = service_request_time[service_request_time['Year']== 2017]
service_time_2017

map_title = alt.TitleParams("2017")
request_time_2017  = alt.Chart(data_geojson_remote, title = map_title).mark_geoshape().transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(service_time_2017, 'name', ['request_time' ,'name', 'neighbourhood_name'])).encode(
    color= alt.Color('request_time:Q', title = "Average days",
                    scale=alt.Scale(scheme = 'blues', zero=False)),
    tooltip= [alt.Tooltip('name:N', title = 'Neighbourhood'),
             alt.Tooltip('request_time:Q', title = 'Request_Time (days)')]
   ).project(type='identity', reflectY=True).properties(
    height = 00, width =500)

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

In [90]:
request_time_2017 | request_time_2023

C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\Users\User\anaconda3\Lib\site-packages\altair\utils\core.py:3

alt.HConcatChart(...)